# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [8]:
import pandas as pd

# Example data: Replace with actual table column names and definitions
data = {
    'table': ['employees', 'departments', 'salary', 'studies'],  # List of table names ### ENTER A TABLE COLUMNS HERE,
    'definition': [ ## ENTER A TABLE DEFINITATIONS HERE
        'Contains employee ID, name, role, department, and salary information.',
        'Details department names and their corresponding department IDs.',
        'Salary details for each year.',
        'Educational studies, name of the institution.'
    ]
}

# Creating a DataFrame from the defined dictionary
df = pd.DataFrame(data)

# Print the DataFrame to verify its structure
print(df)

         table                                         definition
0    employees  Contains employee ID, name, role, department, ...
1  departments  Details department names and their correspondi...
2       salary                      Salary details for each year.
3      studies      Educational studies, name of the institution.


In [9]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [10]:
print(text_tables)

employees: Contains employee ID, name, role, department, and salary information.
departments: Details department names and their corresponding department IDs.
salary: Salary details for each year.
studies: Educational studies, name of the institution.


In [11]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [14]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="Return the name of the best paid employee" ) # "ENTER YOUR QUERY HERE")

In [15]:
print(return_OAI(pqt1))

```json
{
    "tables": ["employees", "salary"]
}
```


In [16]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Return the Education Institution with a higher average salary") #ENTER YOUR QUERY HERE)

In [17]:
print(return_OAI(pqt3))

```json
{
    "tables": ["employees", "salary", "studies"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

# Version 1: Basic Table and Column Setup

In [22]:
pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.


In [34]:
import openai
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def generate_sql_query(description):
    prompt = f"""
    You are an AI capable of generating SQL queries. Use the table names and descriptions to decide which tables are necessary to fulfill the query request.
    
    Table Names and Descriptions:
    - Employers: Contains details about employers such as employer_id and name.
    - Departments: Holds department information connecting employers.
    - Salary: Contains records of salary by employer_id.
    - Studies: Holds educational qualifications linked to employer_id.

    Request: "{description}"
    SQL:
    """
    return chat_with_gpt(prompt)

In [38]:

def chat_with_gpt(prompt):
    try:
        response = openai.Completion.create(  # 'Completion' based on text models
            engine="text-davinci-003",  # Use an appropriate text model like text-davinci-003
            prompt=prompt,
            max_tokens=200,
            temperature=0.7
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return f"Error generating SQL: {e}"


In [36]:
# Test Example
description_1 = "Find employers with a master's degree earning more than $50,000."
result1 = generate_sql_query(description_1)
print(result1)

Error generating SQL: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742



# Version 2: Joining Tables for Comprehensive Data

In [30]:
# User description, maintaining awareness of needed joins
description_2 = "List all departments where the average salary exceeds $40,000."

result2 = generate_sql_query(description_2, tables_info)
print(result2)

NameError: name 'tables_info' is not defined

# Version 3: More Complex Queries with Conditions

In [ ]:
# User description with specific reporting requirements
description_3 = "Show employers who work in the IT department with an average salary decrease last year."

result3 = generate_sql_query(description_3, tables_info)
print(result3)

# Utilizing GPT-3.5 for Generating SQL Queries Based on Table Descriptions

Introduction: This study investigates the potential of GPT-3.5 for generating SQL queries using table names and brief descriptions. The model's proficiency at understanding schema implications and crafting relevant SQL commands was evaluated using several prompt templates of increasing complexity.

Methodology:

Developed multiple prompt structures to test various SQL generation scenarios, ranging from basic to complex queries.
Focused on evaluating the model's accuracy, logical consistency, and creativity in crafting SQL commands purely from descriptive text.
Findings:

Simple Schema Understanding:

Generated SQL for simple requests, successfully linked tables where direct key relations were evident. Output for Description 1 was solid, managing to perform conditional selects effectively.
Intermediate Queries:

Faced challenges when joining multiple tables, particularly when indirect connections existed. While Description 2 produced results, interpretations of nested conditions were occasionally flawed.
Complex Condition Handling:

For multi-table complex queries requiring non-intuitive relations, like Description 3, models misinterpreted intricate conditions. Results didn't encapsulate comprehensive conditions for SQL accuracy.
Conclusion: GPT-3.5 demonstrates a potent start in translating human-readable requests into SQL queries, particularly shining in situations with straightforward single-table accesses and minor logic conditions. However, limitations manifest in multifaceted data relationships and nuanced condition interpretations which would benefit from more detailed metadata or pre-trained schema knowledge.

Lessons Learned:

Structural Limitations: Complex logical conditions and needed joins were weak points, with models occasionally relying on assumptions that led to error.

Benefits of Detailed Prompts: Responses improved remarkably with detailed examples and outlines within prompts, showcasing the need for precise, elaborate input structures.

Limitations of Generalization: Without explicit linkage or examples, generalizations made by the model can lead to incorrect SQL outputs, highlighting reliance on clear schema comprehension.

Recommendations:

Iterative Prompt Enrichment: Develop prompts using multi-turn conversation layers so the model builds context and learns rapidly.
Enhanced Training: Seed with progressive examples of schema that aid in logical structuring, bridging descriptive gaps in SQL order crafting.
Application as an Assisted Tool: Encourage use as a guide alongside domain experts particularly where complex queries are frequent.